[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/121_mamba/121_mamba.ipynb)

# Mamba: El fin de los transformers?

[Mamba](https://arxiv.org/pdf/2312.00752.pdf) es una nueva arquitecutra de red neuronal que promete superar a los transformers, que actualmente sostienen la hegemonía en el *Deep Learning*. Esta arquitectura se inspira en el funcionamiento de las redes neuronales recurrentes, las cuales hemos visto en ateriores posts como [este](https://juansensio.com/blog/034_rnn_intro). Las principales mejores que introduce Mamba son: el uso de *Selective SSM* (los cuales hemos introducido en el post anterior) y una implementación GPU eficiente. Gracias a estas mejoras, Mamba puede ser entrenado de manera eficiente como los Transformers (a diferencia de las redes recurrentes) manteniendo las bondades de las redes recurrentes durante la inferencia, permitiendo trabajar con secuencias de longitud mucho más largas que los transformers. ¿Significa esto el fin de los transformers? En este post descubrirás la respuesta :)

## Redes neuronales recurrentes

Como ya vimos en su momento (https://juansensio.com/blog/034_rnn_intro), las redes neuronales recurrentes permiten el procesamiento de secuencias de datos de longitud arbitraria, lo que las convierte en modelos ideales para trabajar con series temporales o texto.

![rnn](https://miro.medium.com/max/2544/1*aIT6tmnk3qHpStkOX3gGcQ.png)

Estas redes procesan cada *input* de manera independiente, pero mantienen una memoria interna que les permite recordar información de pasos anteriores. 

$$ \mathbf{y}_t = \mathbf{W}_y \mathbf{h}_t = \mathbf{W}_y f(\mathbf{W}_x \mathbf{x}_t + \mathbf{W}_h \mathbf{h}_{t-1}) $$

donde $\mathbf{W}_x$, $\mathbf{W}_h$ y $\mathbf{W}_y$ son matrices de pesos, $\mathbf{x}_t$ es el *input* en el paso $t$, $\mathbf{h}_{t-1}$ es la salida en el paso anterior y $\mathbf{h}_t$ es la salida en el paso $t$. La función $f$ es una función de activación no lineal, como la función sigmoide o la tangente hiperbólica.

Las principales ventajas de las redes recurrentes son su eficiencia durante inferencia, ya que solo necesitan procesar un *input* a la vez, y su capacidad para procesar secuencias de longitud arbitraria (no tienen una *context length*). Sin embargo, estas redes tienen dos grandes desventajas: su entrenamiento en GPU es ineficiente y sus prestaciones son peores que las de los transformers (incluso en sus variantes más potentes como las redes recurrentes LSTM o GRU). Estas desventajas han hecho que las redes recurrentes no sean muy utsadas más allá de problemas sencillos con longitud de secuencia corta.

## El Transformer

También hemos hablado del Transformer en otros posts, como por ejemplo [aquí](https://juansensio.com/blog/063_transformer_encoder) y [aquí](https://juansensio.com/blog/065_transformer_decoder). 

![](https://juansensio.com/blog/065/transformer.png)

Estos modelos se basan en el mecanismo de atención, y es hoy en día la arquitectura *state of the art* en aplicaciones como el procesamiento de lenguaje o la visión por computador. Entre sus ventajas destacan su eficiencia en entrenamiento en GPU y su gran potencia a la hora de modelar relaciones complejas. Sin embargo, su complejidad aumenta de manera cuadrática con la longitud de la secuencia, lo que limita su capacidad para procesar secuencias largas encareciendo el coste computacional durante la inferencia (pese a los muchos esfuerzos para mitigar este problemas como [Flash Attention](https://huggingface.co/docs/text-generation-inference/conceptual/flash_attention) o el cacheado).

## Mamba

Así pues no es de extrañar la investigación en modelos que aúnen las ventajas de redes recurrentes y transformers sin sus desventajas asociadas, y si además dan mejores resultados, mejor que mejor. Mamba representa uno de los primeros esfuerzos en esta dirección.